<a href="https://colab.research.google.com/github/gavhaneswastik/Data-Dreamers/blob/main/Gpt2_text_generation_with_kerasnlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPT2 Text Generation with KerasNLP



In this tutorial, you will learn to use [KerasNLP](https://keras.io/keras_nlp/) to load a
pre-trained Large Language Model (LLM) - [GPT-2 model](https://openai.com/research/better-language-models)
(originally invented by OpenAI), finetune it to a specific text style, and
generate text based on users' input (also known as prompt). You will also learn
how GPT2 adapts quickly to non-English languages, such as Chinese.

##  Before we begin

Colab offers different kinds of runtimes. Make sure to go to **Runtime ->
Change runtime type** and choose the GPU Hardware Accelerator runtime
(which should have >12G host RAM and ~15G GPU RAM) since you will finetune the
GPT-2 model. Running this tutorial on CPU runtime will take hours.

## Install KerasNLP, Choose Backend and Import Dependencies

This examples uses [Keras 3](https://keras.io/keras_3/) to work in any of
`"tensorflow"`, `"jax"` or `"torch"`. Support for Keras 3 is baked into
KerasNLP, simply change the `"KERAS_BACKEND"` environment variable to select
the backend of your choice. We select the JAX backend below.

In [1]:
!pip install git+https://github.com/keras-team/keras-nlp.git -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 44.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 52.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 73.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 101.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 42.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.15.1 requires tensorflow<2.16,>=2.15, but you have tensorflow 2.16.1 which is incompatible.


In [2]:
import os

os.environ["KERAS_BACKEND"] = "jax"  # or "tensorflow" or "torch"

import keras_nlp
import keras
import tensorflow as tf
import time

keras.mixed_precision.set_global_policy("mixed_float16")

## Introduction to Generative Large Language Models (LLMs)

Large language models (LLMs) are a type of machine learning models that are
trained on a large corpus of text data to generate outputs for various natural
language processing (NLP) tasks, such as text generation, question answering,
and machine translation.

Generative LLMs are typically based on deep learning neural networks, such as
the [Transformer architecture](https://arxiv.org/abs/1706.03762) invented by
Google researchers in 2017, and are trained on massive amounts of text data,
often involving billions of words. These models, such as Google [LaMDA](https://blog.google/technology/ai/lamda/)
and [PaLM](https://ai.googleblog.com/2022/04/pathways-language-model-palm-scaling-to.html),
are trained with a large dataset from various data sources which allows them to
generate output for many tasks. The core of Generative LLMs is predicting the
next word in a sentence, often referred as **Causal LM Pretraining**. In this
way LLMs can generate coherent text based on user prompts. For a more
pedagogical discussion on language models, you can refer to the
[Stanford CS324 LLM class](https://stanford-cs324.github.io/winter2022/lectures/introduction/).

## Introduction to KerasNLP

Large Language Models are complex to build and expensive to train from scratch.
Luckily there are pretrained LLMs available for use right away. [KerasNLP](https://keras.io/keras_nlp/)
provides a large number of pre-trained checkpoints that allow you to experiment
with SOTA models without needing to train them yourself.

KerasNLP is a natural language processing library that supports users through
their entire development cycle. KerasNLP offers both pretrained models and
modularized building blocks, so developers could easily reuse pretrained models
or stack their own LLM.

In a nutshell, for generative LLM, KerasNLP offers:

- Pretrained models with `generate()` method, e.g.,
    `keras_nlp.models.GPT2CausalLM` and `keras_nlp.models.OPTCausalLM`.
- Sampler class that implements generation algorithms such as Top-K, Beam and
    contrastive search. These samplers can be used to generate text with
    custom models.

## Load a pre-trained GPT-2 model and generate some text

KerasNLP provides a number of pre-trained models, such as [Google
Bert](https://ai.googleblog.com/2018/11/open-sourcing-bert-state-of-art-pre.html)
and [GPT-2](https://openai.com/research/better-language-models). You can see
the list of models available in the [KerasNLP repository](https://github.com/keras-team/keras-nlp/tree/master/keras_nlp/models).

It's very easy to load the GPT-2 model as you can see below:

In [3]:
# To speed up training and generation, we use preprocessor of length 128
# instead of full length 1024.
preprocessor = keras_nlp.models.GPT2CausalLMPreprocessor.from_preset(
    "gpt2_base_en",
    sequence_length=128,
)
gpt2_lm = keras_nlp.models.GPT2CausalLM.from_preset(
    "gpt2_base_en", preprocessor=preprocessor
)

100%|██████████| 448/448 [00:00<00:00, 383kB/s]
100%|██████████| 0.99M/0.99M [00:01<00:00, 748kB/s]
100%|██████████| 446k/446k [00:01<00:00, 430kB/s]
100%|██████████| 484/484 [00:00<00:00, 375kB/s]
100%|██████████| 475M/475M [00:33<00:00, 14.8MB/s]


Once the model is loaded, you can use it to generate some text right away. Run
the cells below to give it a try. It's as simple as calling a single function
*generate()*:

In [4]:
start = time.time()

output = gpt2_lm.generate("My trip to Yosemite was", max_length=200)
print("\nGPT-2 output:")
print(output)

end = time.time()
print(f"TOTAL TIME ELAPSED: {end - start:.2f}s")


GPT-2 output:
My trip to Yosemite was one of the best experiences of my life. I was able to see the entire area from the top of the summit. The views are incredible and it's a great place to get a little bit of solitude and solitude for the summer. The view of the Yosemite River was incredible. It was amazing how the river flowed. The view is spectacular. It was also amazing how the trees and trees were so tall and so high. I was really happy I was able to see the entire area, and I was very happy I was able to get a little bit of solitude. It is so much better than the previous days, and I really enjoy it. I love it here.

I had a wonderful time at Yosemite and I would like to thank my wife for taking care of me and her family during our trip. She has a great sense of humor about her trip.

I am very excited to share the experience of Yosemite with you all. I am very
TOTAL TIME ELAPSED: 9.79s


Try another one:

In [5]:
start = time.time()

output = gpt2_lm.generate("That Italian restaurant is", max_length=200)
print("\nGPT-2 output:")
print(output)

end = time.time()
print(f"TOTAL TIME ELAPSED: {end - start:.2f}s")


GPT-2 output:
That Italian restaurant is called the 'Italian Restaurant'. It is located in a large building that is surrounded by a large courtyard and the dining room is covered with a huge marble wall. It is a place where you can eat, drink, play, relax, and even go for a drink. It is not a restaurant that is a restaurant.

The Italian Restaurant is not a restaurant, it's just a place that people come and go.

There are a lot of things you can do here, but I would say you should be aware of how to enjoy the place in your life.

I hope this post will help you get started with Italian food and you can also learn more. I hope you like it and if you have any questions, please feel free to leave them below!
TOTAL TIME ELAPSED: 1.79s


Notice how much faster the second call is. This is because the computational
graph is [XLA compiled](https://www.tensorflow.org/xla) in the 1st run and
re-used in the 2nd behind the scenes.

The quality of the generated text looks OK, but we can improve it via
fine-tuning.

## More on the GPT-2 model from KerasNLP

Next up, we will actually fine-tune the model to update its parameters, but
before we do, let's take a look at the full set of tools we have to for working
with for GPT2.

The code of GPT2 can be found
[here](https://github.com/keras-team/keras-nlp/blob/master/keras_nlp/models/gpt2/).
Conceptually the `GPT2CausalLM` can be hierarchically broken down into several
modules in KerasNLP, all of which have a *from_preset()* function that loads a
pretrained model:

- `keras_nlp.models.GPT2Tokenizer`: The tokenizer used by GPT2 model, which is a
    [byte-pair encoder](https://huggingface.co/course/chapter6/5?fw=pt).
- `keras_nlp.models.GPT2CausalLMPreprocessor`: the preprocessor used by GPT2
    causal LM training. It does the tokenization along with other preprocessing
    works such as creating the label and appending the end token.
- `keras_nlp.models.GPT2Backbone`: the GPT2 model, which is a stack of
    `keras_nlp.layers.TransformerDecoder`. This is usually just referred as
    `GPT2`.
- `keras_nlp.models.GPT2CausalLM`: wraps `GPT2Backbone`, it multiplies the
    output of `GPT2Backbone` by embedding matrix to generate logits over
    vocab tokens.

## Finetune on imdb_reviews dataset

Now you have the knowledge of the GPT-2 model from KerasNLP, you can take one
step further to finetune the model so that it generates text in a specific
style, short or long, strict or casual. In this tutorial, we will use imdb_reviews dataset for example.

Let's take a look inside sample data from the imdb_reviews TensorFlow Dataset. There
are two features:

- **__document__**: text of the post.
- **__title__**: the title.

In [6]:
import tensorflow_datasets as tfds

imdb_reviews_ds = tfds.load("imdb_reviews", split="train", as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.C2KITA_1.0.0/imdb_reviews-train.tfrecor…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.C2KITA_1.0.0/imdb_reviews-test.tfrecord…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.C2KITA_1.0.0/imdb_reviews-unsupervised.…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


Let's take a look inside sample data from the reddit TensorFlow Dataset. There are two features:

document: text of the post.
title: the title.

In [7]:

for document, title in imdb_reviews_ds:
    print(document.numpy())
    print(title.numpy())
    break

b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
0


In our case, we are performing next word prediction in a language model, so we only need the 'document' feature.

In [8]:
train_ds = (
    imdb_reviews_ds.map(lambda document, _: document)
    .batch(32)
    .cache()
    .prefetch(tf.data.AUTOTUNE)
)

Now you can finetune the model using the familiar fit() function. Note that preprocessor will be automatically called inside fit method since GPT2CausalLM is a keras_nlp.models.Task instance.

This step takes quite a bit of GPU memory and a long time if we were to train it all the way to a fully trained state. Here we just use part of the dataset for demo purposes

Create TensorFlow Datasets from the Preprocessed Data

In [9]:
train_ds = train_ds.take(500)
num_epochs = 5

# Linearly decaying learning rate.
learning_rate = keras.optimizers.schedules.PolynomialDecay(
    5e-5,
    decay_steps=train_ds.cardinality() * num_epochs,
    end_learning_rate=0.0,
)
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
gpt2_lm.compile(
    optimizer=keras.optimizers.Adam(learning_rate),
    loss=loss,
    weighted_metrics=["accuracy"],
)

gpt2_lm.fit(train_ds, epochs=num_epochs)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 203s 344ms/step - accuracy: 0.3226 - loss: 3.6336
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 150s 272ms/step - accuracy: 0.3421 - loss: 3.4345
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 143s 284ms/step - accuracy: 0.3553 - loss: 3.3424
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 143s 284ms/step - accuracy: 0.3615 - loss: 3.2881
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 143s 284ms/step - accuracy: 0.3656 - loss: 3.2541


After fine-tuning is finished, you can again generate text using the same generate() function. This time, the text will be closer to writing style, and the generated length will be close to our preset length in the training set.

In [10]:
start = time.time()

output = gpt2_lm.generate("I like basketball", max_length=200)
print("\nGPT-2 output:")
print(output)

end = time.time()
print(f"TOTAL TIME ELAPSED: {end - start:.2f}s")



GPT-2 output:
I like basketball. I love the game of basketball. I love the movie "Dirty Harry." I love the movie "The Big Chill." I love the movie "The Big Chill." I love the movie "The Big Chill." I love the movie "The Big Chill." I love the movie "The Big Chill."
TOTAL TIME ELAPSED: 6.79s


After fine-tuning is finished, you can again generate text using the same generate() function. This time, the text will be closer to imdb_reviews writing style, and the generated length will be close to our preset length in the training set.

## Into the Sampling Method

In KerasNLP, we offer a few sampling methods, e.g., contrastive search,
Top-K and beam sampling. By default, our `GPT2CausalLM` uses Top-k search, but
you can choose your own sampling method.

Much like optimizer and activations, there are two ways to specify your custom
sampler:

- Use a string identifier, such as "greedy", you are using the default
configuration via this way.
- Pass a `keras_nlp.samplers.Sampler` instance, you can use custom configuration
via this way.

In [11]:
# Use a string identifier.
gpt2_lm.compile(sampler="top_k")
output = gpt2_lm.generate("the movie was fantastic  beccause", max_length=200)
print("\nGPT-2 output:")
print(output)

# Use a `Sampler` instance. `GreedySampler` tends to repeat itself,
greedy_sampler = keras_nlp.samplers.GreedySampler()
gpt2_lm.compile(sampler=greedy_sampler)

output = gpt2_lm.generate("the storyline was gripping and", max_length=200)
print("\nGPT-2 output:")
print(output)


GPT-2 output:
the movie was fantastic  beccause it has a story, it has a good cast , i think it is the best movie i have seen in a long time. the movie was great i think i will watch it again and again and i would like to watch it more. it is the best movie i have seen in a long time.

GPT-2 output:
the storyline was gripping and the characters were believable. the acting was good and the story was well told. the story was about a young man who is a member of a small town who is attacked by a group of thugs. he is saved by a group of thugs who are trying to kill him. the story is about a young man who is attacked by a group of thugs who are trying to kill him. he is saved by a group of thugs who are trying to kill him. the story is about a young man who is attacked by a group of thugs


For more details on KerasNLP `Sampler` class, you can check the code
[here](https://github.com/keras-team/keras-nlp/tree/master/keras_nlp/samplers).

##Finetune on Chinese Poem Dataset
We can also finetune GPT2 on non-English datasets. For readers knowing Chinese, this part illustrates how to fine-tune GPT2 on Chinese poem dataset to teach our model to become a poet!

Because GPT2 uses byte-pair encoder, and the original pretraining dataset contains some Chinese characters, we can use the original vocab to finetune on Chinese dataset.

In [12]:
!# Load chinese poetry dataset.
!git clone https://github.com/chinese-poetry/chinese-poetry.git

Cloning into 'chinese-poetry'...
remote: Enumerating objects: 7323, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 7323 (delta 5), reused 10 (delta 3), pack-reused 7309
Receiving objects: 100% (7323/7323), 236.98 MiB | 11.16 MiB/s, done.
Resolving deltas: 100% (5003/5003), done.
Updating files: 100% (2285/2285), done.


Load text from the json file. We only use《全唐诗》for demo purposes.

In [13]:
import os
import json

poem_collection = []
for file in os.listdir("chinese-poetry/全唐诗"):
    if ".json" not in file or "poet" not in file:
        continue
    full_filename = "%s/%s" % ("chinese-poetry/全唐诗", file)
    with open(full_filename, "r") as f:
        content = json.load(f)
        poem_collection.extend(content)

paragraphs = ["".join(data["paragraphs"]) for data in poem_collection]

Let's take a look at sample data.

In [14]:
print(paragraphs[0])

時闢西湖作勝遊，使君元是濟川舟。


Similar as imbd_dataset example, we convert to TF dataset, and only use partial data
to train.

In [15]:
train_ds = (
    tf.data.Dataset.from_tensor_slices(paragraphs)
    .batch(16)
    .cache()
    .prefetch(tf.data.AUTOTUNE)
)

# Running through the whole dataset takes long, only take `500` and run 1
# epochs for demo purposes.
train_ds = train_ds.take(500)
num_epochs = 1

learning_rate = keras.optimizers.schedules.PolynomialDecay(
    5e-4,
    decay_steps=train_ds.cardinality() * num_epochs,
    end_learning_rate=0.0,
)
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
gpt2_lm.compile(
    optimizer=keras.optimizers.Adam(learning_rate),
    loss=loss,
    weighted_metrics=["accuracy"],
)

gpt2_lm.fit(train_ds, epochs=num_epochs)

500/500 ━━━━━━━━━━━━━━━━━━━━ 114s 173ms/step - accuracy: 0.2399 - loss: 2.8214


Let's check the result!

In [16]:
output = gpt2_lm.generate("万字符是很酷的男孩", max_length=200)
print(output)

万字符是很酷的男孩，那淚山聲那淚。風爛山風山除，陶更風臥紅頻。


##Finetune titanic dataset
Now you have the knowledge of the GPT-2 model from KerasNLP, you can take one step further to finetune the model so that it generates text in a specific style, short or long, strict or casual. In this tutorial, we will use game of thrones
 dataset for example.

Let's take a look inside sample data from the titanic Dataset. There are two features:

document: text of the post.
title: the title.

In [19]:
import json # Import the json module


with open('/titanic.json', 'r', encoding='utf-8') as f:
    titanic_data = json.load(f) # Now you can use json.load()

In [20]:
# Create a TensorFlow dataset from the JSON data

texts = titanic_data["text"]
titanic_ds = tf.data.Dataset.from_tensor_slices(texts)

In [21]:
# Prepare the dataset for training
train_ds = (
    titanic_ds
    .batch(32)
    .cache()
    .prefetch(tf.data.AUTOTUNE)
)

Now you can finetune the model using the familiar fit() function. Note that preprocessor will be automatically called inside fit method since GPT2CausalLM is a keras_nlp.models.Task instance.

This step takes quite a bit of GPU memory and a long time if we were to train it all the way to a fully trained state. Here we just use part of the dataset for demo purposes

In [25]:
# Take a subset for training (optional, depending on your needs)
train_ds = train_ds.take(500)
num_epochs = 5

# Configure a dynamically decreasing learning rate based on linear decay
learning_rate = keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=5e-5,
    decay_steps=train_ds.cardinality() * num_epochs,
    end_learning_rate=0.0,
)

In [26]:
# Define the loss function
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Assuming gpt2_lm is your GPT-2 model
# If not already defined, you need to load or create your GPT-2 model here

# Model compilation
gpt2_lm.compile(
    optimizer=keras.optimizers.Adam(learning_rate),
    loss=loss,
    weighted_metrics=["accuracy"],
)

# Train the model
gpt2_lm.fit(train_ds, epochs=num_epochs)

Epoch 1/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 52s 32s/step - accuracy: 0.0203 - loss: 1.3276
Epoch 2/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 16s 255ms/step - accuracy: 0.1021 - loss: 0.9444
Epoch 3/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 284ms/step - accuracy: 0.2102 - loss: 0.7704
Epoch 4/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 250ms/step - accuracy: 0.2593 - loss: 0.6954
Epoch 5/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 281ms/step - accuracy: 0.2566 - loss: 0.6675


After fine-tuning is finished, you can again generate text using the same generate() function. This time, the text will be closer to  writing style, and the generated length will be close to our preset length in the training set.

In [27]:
# Inference
start = time.time()
output= gpt2_lm.generate("I'll  ", max_length=100)
print("\nGPT-2 output:")
print(output)
end = time.time()
print(f"TOTAL TIME ELAPSED: {end - start:.2f}s")


GPT-2 output:
I'll   to be honest, I'm really glad that this is the best way to describe this movie and it's really not that bad. I'm not a big fan of the way it is, but I've never been so excited for this movie.
TOTAL TIME ELAPSED: 8.53s


#Finetune realtime dataset
Now you have the knowledge of the GPT-2 model from KerasNLP, you can take one step further to finetune the model so that it generates text in a specific style, short or long, strict or casual. In this tutorial, we will use realtime dataset for example.

Let's take a look inside sample data from the realtime Dataset. There are two features:

document: text of the text transcription. title: the title.

In [30]:
import json
with open("/content/realtime_dataset.json", 'r', encoding='utf-8') as f:
    realtime_data=json.load(f)

In [32]:
# Extract texts from the dataset
texts = [realtime_data.get('text', '')]  # Access the 'text' value directly, or provide an empty string if it doesn't exist

# Create a TensorFlow dataset from the texts
realtime_ds = tf.data.Dataset.from_tensor_slices(texts)

# Prepare the dataset for training
train_ds = (
    realtime_ds
    .batch(32)
    .cache()
    .prefetch(tf.data.AUTOTUNE)
)

Now you can finetune the model using the familiar fit() function. Note that preprocessor will be automatically called inside fit method since GPT2CausalLM is a keras_nlp.models.Task instance.

This step takes quite a bit of GPU memory and a long time if we were to train it all the way to a fully trained state. Here we just use part of the dataset for demo purposes





In [33]:
# Take a subset for training (optional, depending on your needs)
train_ds = train_ds.take(100)
num_epochs = 10

# Configure a dynamically decreasing learning rate based on linear decay
learning_rate = keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=5e-5,
    decay_steps=train_ds.cardinality() * num_epochs,
    end_learning_rate=0.0,
)

# Define the loss function
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Model compilation
gpt2_lm.compile(
    optimizer=keras.optimizers.Adam(learning_rate),
    loss=loss,
    weighted_metrics=["accuracy"],
)

# Train the model
gpt2_lm.fit(train_ds, epochs=num_epochs)

Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 25s 25s/step - accuracy: 0.2500 - loss: 4.9051
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 17s 17s/step - accuracy: 0.2578 - loss: 4.8984
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.2422 - loss: 4.8969
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.3047 - loss: 4.1319
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step - accuracy: 0.3750 - loss: 3.5146
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.4297 - loss: 3.2238
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.4844 - loss: 2.7783
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.4844 - loss: 2.6703
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.5000 - loss: 2.5770
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.5391 - loss: 2.4814



After fine-tuning is finished, you can again generate text using the same generate() function. This time, the text will be closer to writing  style, and the generated length will be close to our preset length in the training set.
```



In [34]:
# Inference
start = time.time()
output = gpt2_lm.generate("He has to", max_length=200)
print("\nGPT-2 output:")
print(output)
end = time.time()
print(f"TOTAL TIME ELAPSED: {end-start:.2f}s")


GPT-2 output:
He has to be in the first movie. I don't see that in the first. It's in my second one. It's in my second one. It was in the first movie. I can't even see that in the first one. I'm in the second one.
TOTAL TIME ELAPSED: 6.96s
